In [ ]:
from google.colab import files

uploaded = files.upload()


KeyboardInterrupt: 

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

conn = sqlite3.connect("/content/tiny_data.db")

In [ ]:
# ---------- Leitura das tabelas ----------
tabelas = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("Tabelas disponíveis:")
display(tabelas)

vendas = pd.read_sql("SELECT * FROM vendas", conn)
produtos = pd.read_sql("SELECT * FROM produtos", conn)
clientes = pd.read_sql("SELECT * FROM clientes", conn)
# Assuming 'tiny_merged' table exists based on the original code
try:
    merged = pd.read_sql("SELECT * FROM tiny_merged", conn)
    print(f"{len(vendas)} vendas | {len(produtos)} itens | {len(clientes)} clientes | {len(merged)} merged records")
except pd.io.sql.DatabaseError as e:
    print(f"Could not read 'tiny_merged' table: {e}")
    print(f"{len(vendas)} vendas | {len(produtos)} itens | {len(clientes)} clientes")

In [ ]:
# ---------- Pré-processamento ----------
vendas['data_emissao'] = pd.to_datetime(vendas['data_emissao'], errors='coerce')
vendas['mes'] = vendas['data_emissao'].dt.to_period('M')

# ---------- Faturamento Mensal ----------
faturamento = vendas.groupby('mes')['valor_total'].sum().reset_index()
faturamento['mes'] = faturamento['mes'].astype(str)

plt.figure(figsize=(10,5))
sns.barplot(data=faturamento, x='mes', y='valor_total', color='steelblue')
plt.xticks(rotation=45)
plt.title('Faturamento Mensal (R$)')
plt.xlabel('Mês')
plt.ylabel('Valor Total (R$)')
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

In [ ]:
# Garantir que todos os valores são numéricos
vendas['valor_total'] = pd.to_numeric(vendas['valor_total'], errors='coerce')

# Recalcular agrupamento para garantir integridade
vendas_mes = vendas.groupby('mes').agg({'valor_total':'sum','numero_nota':'count'}).reset_index()

# Ticket médio como float
vendas_mes['ticket_medio'] = vendas_mes['valor_total'] / vendas_mes['numero_nota']
vendas_mes['ticket_medio'] = pd.to_numeric(vendas_mes['ticket_medio'], errors='coerce')


In [ ]:
# Recalcular agrupamento com limpeza explícita
vendas['valor_total'] = pd.to_numeric(vendas['valor_total'], errors='coerce')

# Converter data_emissao corretamente
vendas['data_emissao'] = pd.to_datetime(vendas['data_emissao'], errors='coerce')

# Criar coluna "mes" como string simples (não Period)
vendas['mes'] = vendas['data_emissao'].dt.strftime('%Y-%m')

# Agrupar por mês
vendas_mes = vendas.groupby('mes', as_index=False).agg({
    'valor_total': 'sum',
    'numero_nota': 'count'
})

# Calcular ticket médio
vendas_mes['ticket_medio'] = vendas_mes['valor_total'] / vendas_mes['numero_nota']

# Garantir tipos numéricos puros
vendas_mes['ticket_medio'] = pd.to_numeric(vendas_mes['ticket_medio'], errors='coerce')
vendas_mes['valor_total'] = pd.to_numeric(vendas_mes['valor_total'], errors='coerce')
vendas_mes['numero_nota'] = pd.to_numeric(vendas_mes['numero_nota'], errors='coerce')

# --- Plot limpo ---

plt.figure(figsize=(10,5))
sns.lineplot(
    data=vendas_mes.sort_values('mes'),
    x='mes',
    y='ticket_medio',
    marker='o',
    color='darkorange'
)
plt.title('Ticket Médio por Mês')
plt.xlabel('Mês')
plt.ylabel('Ticket Médio (R$)')
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

# Mostrar tabela pra confirmar
display(vendas_mes.head(12))


In [ ]:
# ---------- Curva ABC de Produtos ----------
produtos_vendas = merged.groupby('nome_produto')['valor_total_item'].sum().sort_values(ascending=False).reset_index()
produtos_vendas['perc'] = produtos_vendas['valor_total_item'] / produtos_vendas['valor_total_item'].sum() * 100
produtos_vendas['perc_acum'] = produtos_vendas['perc'].cumsum()

plt.figure(figsize=(10,5))
sns.lineplot(data=produtos_vendas, x=produtos_vendas.index, y='perc_acum', color='seagreen')
plt.title('Curva ABC de Produtos')
plt.xlabel('Ranking de Produtos')
plt.ylabel('Participação Acumulada (%)')
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

In [ ]:
# Top 20 produtos mais vendidos (em valor)
top20 = produtos_vendas.head(20)

plt.figure(figsize=(10,6))
sns.barplot(data=top20, x='valor_total_item', y='nome_produto', palette='viridis')
plt.title('Top 20 Produtos por Faturamento')
plt.xlabel('Faturamento (R$)')
plt.ylabel('Produto')
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

display(top20)


In [ ]:
# ---------- Top 10 Cidades ----------
clientes_cidade = clientes.groupby('cidade')['id_nota'].count().sort_values(ascending=False).head(10)
clientes_cidade.plot(kind='bar', figsize=(10,5), color='mediumpurple')
plt.title('Top 10 Cidades com Mais Clientes')
plt.ylabel('Qtde de Notas')
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.show()

In [ ]:
# ---------- Faturamento por UF ----------
merged['valor_total_item'] = pd.to_numeric(merged['valor_total_item'], errors='coerce')
fat_uf = merged.groupby('uf')['valor_total_item'].sum().sort_values(ascending=False).reset_index()

plt.figure(figsize=(10,5))
sns.barplot(data=faturamento, x='mes', y='valor_total', palette='Blues_d')
plt.xticks(rotation=45)
plt.title('Faturamento Mensal (R$)')
plt.xlabel('Mês')
plt.ylabel('Valor Total (R$)')
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()

conn.close()

In [ ]:
# ============================================================
# 📦 EXPORTAÇÃO DOS RESULTADOS ANALÍTICOS (Fase 3)
# ============================================================

import pandas as pd
from google.colab import files

# ============================================================
# 🔹 FATURAMENTO MENSAL
# ============================================================
faturamento_mensal = vendas.groupby('mes', as_index=False)['valor_total'].sum()
faturamento_mensal.rename(columns={'valor_total': 'faturamento_total'}, inplace=True)

# ============================================================
# 🔹 TICKET MÉDIO MENSAL
# ============================================================
vendas_mes = vendas.groupby('mes', as_index=False).agg({
    'valor_total': 'sum',
    'numero_nota': 'count'
})
vendas_mes['ticket_medio'] = vendas_mes['valor_total'] / vendas_mes['numero_nota']
ticket_medio_mensal = vendas_mes[['mes', 'ticket_medio']]

# ============================================================
# 🔹 TOP PRODUTOS (Curva ABC)
# ============================================================
produtos_vendas = merged.groupby('nome_produto', as_index=False)['valor_total_item'].sum()
produtos_vendas = produtos_vendas.sort_values('valor_total_item', ascending=False)
produtos_vendas['perc'] = produtos_vendas['valor_total_item'] / produtos_vendas['valor_total_item'].sum() * 100
produtos_vendas['perc_acum'] = produtos_vendas['perc'].cumsum()

# Classificação ABC
def classificar_abc(x):
    if x <= 80:
        return 'A'
    elif x <= 95:
        return 'B'
    else:
        return 'C'
produtos_vendas['classe_abc'] = produtos_vendas['perc_acum'].apply(classificar_abc)

# ============================================================
# 🔹 FATURAMENTO POR ESTADO (UF)
# ============================================================
merged['valor_total_item'] = pd.to_numeric(merged['valor_total_item'], errors='coerce')
fat_uf = merged.groupby('uf', as_index=False)['valor_total_item'].sum()
fat_uf.rename(columns={'valor_total_item': 'faturamento_total'}, inplace=True)
fat_uf = fat_uf.sort_values('faturamento_total', ascending=False)

# ============================================================
# 🔹 CLIENTES POR ESTADO
# ============================================================
clientes_por_estado = clientes.groupby('uf', as_index=False)['id_nota'].count()
clientes_por_estado.rename(columns={'id_nota': 'quantidade_clientes'}, inplace=True)

# ============================================================
# 💾 EXPORTAR TODOS OS RESULTADOS PARA CSV
# ============================================================
arquivos = {
    "faturamento_mensal.csv": faturamento_mensal,
    "ticket_medio_mensal.csv": ticket_medio_mensal,
    "produtos_vendas.csv": produtos_vendas,
    "clientes_por_estado.csv": clientes_por_estado,
    "faturamento_por_estado.csv": fat_uf
}

for nome, df in arquivos.items():
    df.to_csv(nome, index=False, encoding='utf-8')
    print(f"[OK] {nome} salvo ({len(df)} linhas)")
    files.download(nome)


# FASE 2


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [ ]:
# Configurações visuais
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("dark:#5A9_r")

# Caminho base
BASE_DIR = Path("/content/processados")
df = pd.read_csv(BASE_DIR / "dados_gerais.csv")

In [ ]:
print("Dimensões da base:", df.shape)
display(df.head(3))

Dimensões da base: (701, 44)


,sku,produto,vendas,valor_total_market,visualizacoes,devolucoes,ano,mes,canal,id_nota,...,cpf_cnpj,endereco,numero,bairro,cidade,uf,cep,receita_total,receita_tiny,faturamento_total_canais
0,NAN,NaN,NaN,NaN,NaN,NaN,2024,NaN,amazon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,NAN,NaN,NaN,NaN,NaN,NaN,2024,NaN,amazon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2,NAN,NaN,NaN,NaN,NaN,NaN,2024,NaN,amazon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
